In [1]:
import torch
import numpy as np

In [4]:
y = torch.zeros(1)
y

tensor([0.])

In [13]:
y.random_()

tensor([12284051.])

In [14]:
# get specific class from ImageNet Val set

In [68]:
from pathlib import Path
base_dir = Path('/mnt/lustre/share/images')
val_meta_list = base_dir / 'meta/' / 'val.txt'

In [40]:
with open(val_meta_list) as val:
    val_imgs = [item.split() for item in val.readlines()]

In [52]:
from collections import defaultdict
label_map_imgs = defaultdict(list)

In [57]:
from tqdm import tqdm
for pair in tqdm(val_imgs[:]):
    label_map_imgs[int(pair[1])].append(pair[0])

100%|██████████| 50000/50000 [00:00<00:00, 893317.43it/s]


In [71]:
cars = label_map_imgs[817][:10]

In [72]:
tgt_dir = '/mnt/lustre/yslan/Repo/Generation/deep-generative-prior/data/cars/sports'

In [73]:
import shutil

In [74]:
for item in cars:
    shutil.copy(base_dir / 'val' / item, tgt_dir)

## change pretrained model weights

In [46]:
import torch
from collections import OrderedDict

def key_transformation_stylegan_d(key):
    if key.startswith('conv_body'):
        return key.replace('conv_body', 'convs')
    return key

def key_transformation_stylegan_g(key):
    if key.startswith('style_mlp'):
        return key.replace('style_mlp', 'style')
    if key.startswith('style_convs'):
        return key.replace('style_convs', 'convs')
    if key.startswith('style_conv1'):
        return key.replace('style_conv1', 'conv1')
    return key

def rename_state_dict_keys(state_dict, key_transformation, target=None):
    """
    source             -> loaded Source dict 
    key_transformation -> Function that accepts the old key names of the state
                          dict as the only argument and returns the new key name.
    target (optional)  -> Path at which the new state dict should be saved
                          (defaults to `source`)
    Example:
    Rename the key `layer.0.weight` `layer.1.weight` and keep the names of all
    other keys.
    ```py
    def key_transformation(old_key):
        if old_key == "layer.0.weight":
            return "layer.1.weight"
        return old_key
    rename_state_dict_keys(state_dict_path, key_transformation)
    ```
    """
  
#     state_dict = torch.load(source)
    new_state_dict = OrderedDict()

    for key, value in state_dict.items():
        new_key = key_transformation(key)
        new_state_dict[new_key] = value

    torch.save(new_state_dict, target)


In [47]:
import torch
stylegan_g = '/mnt/lustre/yslan/Repo/NVS/Projects/Generative/stylegan2-pytorch/checkpoint/pretrained/stylegan2_car_config_f_512_official-e8fcab4f.pth'

d = '/mnt/lustre/yslan/Repo/NVS/Projects/Generative/stylegan2-pytorch/checkpoint/pretrained/stylegan2_car_config_f_512_discriminator_official-5008e3d1.pth'
d = torch.load(d)

In [54]:
# stylegan_g = torch.load(stylegan_g)['params_ema']
stylegan_g = torch.load('./pretrained/stylegan2/stylegan2_car_512_g.pth')
stylegan_g.keys()

odict_keys(['style.1.weight', 'style.1.bias', 'style.2.weight', 'style.2.bias', 'style.3.weight', 'style.3.bias', 'style.4.weight', 'style.4.bias', 'style.5.weight', 'style.5.bias', 'style.6.weight', 'style.6.bias', 'style.7.weight', 'style.7.bias', 'style.8.weight', 'style.8.bias', 'constant_input.weight', 'conv1.weight', 'conv1.modulated_conv.weight', 'conv1.modulated_conv.modulation.weight', 'conv1.modulated_conv.modulation.bias', 'conv1.activate.bias', 'to_rgb1.bias', 'to_rgb1.modulated_conv.weight', 'to_rgb1.modulated_conv.modulation.weight', 'to_rgb1.modulated_conv.modulation.bias', 'convs.0.weight', 'convs.0.modulated_conv.weight', 'convs.0.modulated_conv.modulation.weight', 'convs.0.modulated_conv.modulation.bias', 'convs.0.activate.bias', 'convs.1.weight', 'convs.1.modulated_conv.weight', 'convs.1.modulated_conv.modulation.weight', 'convs.1.modulated_conv.modulation.bias', 'convs.1.activate.bias', 'convs.2.weight', 'convs.2.modulated_conv.weight', 'convs.2.modulated_conv.modul

In [52]:
rename_state_dict_keys(stylegan_g, key_transformation_stylegan_g, 'stylegan2_car_512_g.pth')
# rename_state_dict_keys(d, key_transformation_stylegan_d, 'stylegan2_car_512_d.pth')

In [6]:
from collections import OrderedDict

In [34]:
# stylegan_g

TypeError: 'odict_keys' object is not subscriptable

* style_mlp -> style
* style_conv -> convs
* style_conv1 -> conv1

In [23]:
s

dict_keys(['params'])

In [28]:

torch.rand((3,4))[:,None][0].shape

torch.Size([1, 4])